# Analize result

In [1]:
import json
import numpy as np
import pandas as pd 
import os

from generator import CatmGenerator
from models_tf.CADM import CADM

from keras import optimizers, losses, metrics
import tensorflow as tf
from tensorflow.data import Dataset

model_saved_dir = "saved_models/cluster/cadm_catm_env1"
dataset_dir = "dataset/DAM_nonToF/all0508"
model_weights_file="checkpoint_70"
device = '/CPU:0'
lr = 0.00001

2022-05-20 14:08:56.357900: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open(os.path.join(model_saved_dir, 'info.json'), 'r') as f:
    info = json.load(f)

print("keys in info:")
for key in info.keys():
    print(key)

keys in info:
data_set
train_list
test_list
model_name
input_shape
d_model


In [3]:
#create dataset
img_size = (info["input_shape"][1], info["input_shape"][2], info["input_shape"][3])
data_gen = CatmGenerator(info["test_list"], dataset_dir, img_resize=(img_size[0], img_size[1]))
n_batch_size = 32
T_MAX = data_gen.sequence_len()
N_MOTION = info["d_model"]
data_list = info["test_list"]
with tf.device(device):
    dataset = Dataset.from_generator(
        data_gen.generator,
        output_signature=(
            tf.TensorSpec(shape=(T_MAX, img_size[0], img_size[1], img_size[2]), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

2022-05-20 14:08:57.407608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-20 14:08:57.435924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-20 14:08:57.436078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-20 14:08:57.436597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [4]:
#load model

with tf.device(device):
    model = CADM(N_MOTION)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy()],
        run_eagerly=True
    )
    
    model.load_weights(os.path.join(model_saved_dir, model_weights_file)) 


In [5]:
# evaluate output values
with tf.device(device):
    predicted = []
    labels = []
    users = []
    for index, (data, label) in enumerate(dataset):
        assert int(data_list[index].split('-')[1])-1 == int(label)
        user = data_list[index].split('-')[0]
        data = tf.expand_dims(data, axis=0)
        p = model(data) #(1, 100, 10)
        p = tf.math.argmax(p, axis=-1)
        p = tf.squeeze(p)
        
        predicted.append(int(p))
        labels.append(int(label))
        users.append(user)
        
result = pd.DataFrame({
    "Users":users,
    "Labels":labels,
    "Predicted":predicted})  

2022-05-20 14:09:06.386347: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 65.44M (68616192 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


     Users  Labels  Predicted
0    user1       6          6
1    user4       4          3
2    user2       6          6
3    user4       6          6
4    user5       2          2
..     ...     ...        ...
262  user1       1          1
263  user5       3          3
264  user1       6          6
265  user1       3          3
266  user5       7          7

[267 rows x 3 columns]


In [23]:
user_split = []
for i in range(5):
    user_split.append(result[result.Users == ("user"+str(i+1))])

In [24]:
for u, df in enumerate(user_split):
    print("--------->user{}".format(u+1))
    for m in range(8):
        print("\t------>motion{}---->".format(m))
        
        label_count = len(df[df.Labels==m])
        
        tmp = df[df.Labels==m]
        tmp = tmp[tmp.Labels==tmp.Predicted]
        label_right_count = len(tmp)
        
        p = label_right_count/label_count * 100
        
        print('total: {}, accuracy: {}, percent: {}'.format(label_count, label_right_count, p))

--------->user1
	------>motion0---->
total: 7, accuracy: 6, percent: 85.71428571428571
	------>motion1---->
total: 8, accuracy: 7, percent: 87.5
	------>motion2---->
total: 4, accuracy: 3, percent: 75.0
	------>motion3---->
total: 6, accuracy: 5, percent: 83.33333333333334
	------>motion4---->
total: 9, accuracy: 8, percent: 88.88888888888889
	------>motion5---->
total: 7, accuracy: 7, percent: 100.0
	------>motion6---->
total: 10, accuracy: 9, percent: 90.0
	------>motion7---->
total: 6, accuracy: 5, percent: 83.33333333333334
--------->user2
	------>motion0---->
total: 7, accuracy: 7, percent: 100.0
	------>motion1---->
total: 5, accuracy: 4, percent: 80.0
	------>motion2---->
total: 8, accuracy: 7, percent: 87.5
	------>motion3---->
total: 8, accuracy: 7, percent: 87.5
	------>motion4---->
total: 7, accuracy: 5, percent: 71.42857142857143
	------>motion5---->
total: 8, accuracy: 8, percent: 100.0
	------>motion6---->
total: 4, accuracy: 4, percent: 100.0
	------>motion7---->
total: 

In [26]:
for m in range(8):
    print('------->motion{}'.format(m))
    label_count = len(result[result.Labels==m])

    tmp = result[result.Labels==m]
    tmp = tmp[tmp.Labels==tmp.Predicted]
    label_right_count = len(tmp)
    p = label_right_count/label_count * 100
    print('total: {}, accuracy: {}, percent{}'.format(label_count, label_right_count, p))

------->motion0
total: 31, accuracy: 28, percent90.32258064516128
------->motion1
total: 28, accuracy: 22, percent78.57142857142857
------->motion2
total: 35, accuracy: 32, percent91.42857142857143
------->motion3
total: 31, accuracy: 26, percent83.87096774193549
------->motion4
total: 32, accuracy: 27, percent84.375
------->motion5
total: 40, accuracy: 34, percent85.0
------->motion6
total: 36, accuracy: 31, percent86.11111111111111
------->motion7
total: 34, accuracy: 29, percent85.29411764705883


In [5]:
# evaluate loss and accuracy
dataset_batched = dataset.batch(n_batch_size)
with tf.device(device):
    loss = model.evaluate(x=dataset)
    print(loss)

2022-05-20 13:39:34.987667: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 65.44M (68616192 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


9/9 [==============================] - 51s 6s/step - loss: 0.4203 - sparse_categorical_accuracy: 0.8577
[0.42025670409202576, 0.8576778769493103]
